<a href="https://colab.research.google.com/github/Valeeew/Data_management_of_an_online_store/blob/master/Data_management_of_an_online_store.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Management & Optimisation d'une boutique en ligne
## Par Valentin MONTEIRO 
---
### Sommaire
> ### IMPORT
- [Import des library](#scrollTo=tAgUgPb2dUt3&line=1&uniqifier=1)
- [Import des DATA](#scrollTo=LGGkJyQgd2Vc&line=3&uniqifier=1)  
- [Lexique des données](#scrollTo=oG89DAzeosCW&line=1&uniqifier=1)


## IMPORT

---
> ### Import des Libraries
---



In [5]:
import pandas as pd

---
> ### Import des DATA from Github
---




In [12]:
# Clone the entire repo & import DATA
!git clone -l -s git://github.com/Valeeew/Data_management_of_an_online_store cloned-repo
%cd cloned-repo
!ls

# Enregistrement des DATA dans les DATAframe
df_erp = pd.read_excel("erp.xlsx")
df_liaison = pd.read_excel("liaison.xlsx")
df_web = pd.read_excel("web.xlsx")

Cloning into 'cloned-repo'...
remote: Enumerating objects: 6, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 6 (delta 1), reused 6 (delta 1), pack-reused 0
Receiving objects: 100% (6/6), 339.06 KiB | 24.22 MiB/s, done.
Resolving deltas: 100% (1/1), done.
/content/cloned-repo/cloned-repo/cloned-repo/cloned-repo
erp.xlsx  liaison.xlsx	web.xlsx


# Lexique des données

## df_erp


error: src refspec main does not match any.
error: failed to push some refs to 'git://github.com/Valeeew/Data_management_of_an_online_store'
